# Kundur Two Areas

### Import Libraries

In [ ]:
import subprocess, sys, os
import urllib.request

dpsim_root_dir = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
sys.path.insert(0, os.path.join(dpsim_root_dir, 'python/src/dpsim/'))
sys.path.insert(0, os.path.join(dpsim_root_dir, 'build'))

import matpower
import dpsimpy
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import urllib.request
import matplotlib.pyplot as plt

### Get simulation data

In [ ]:
if not os.path.exists('Kundur2Areas-data'):
    os.mkdir('Kundur2Areas-data')

url_static = 'https://github.com/martinmoraga/dpsim_data/raw/main/Kundur2Areas/matpower/Kundur2Areas.mat'
url_dynamic = 'https://github.com/martinmoraga/dpsim_data/raw/main/Kundur2Areas/matpower/Kundur2Areas_dyn.mat'
local_file_static = './Kundur2Areas-data/Kundur2Areas.mat'
local_file_dynamic = './Kundur2Areas-data/Kundur2Areas_dyn.mat'
urllib.request.urlretrieve(url_static, local_file_static)
urllib.request.urlretrieve(url_dynamic, local_file_dynamic)

### 1. Powerflow for initialization

In [ ]:
sim_name_pf = 'Kundur2Areas_PF'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)

# read and create dpsim topology
mpc_reader_pf = matpower.Reader(mpc_file_path=local_file_static, mpc_name='Kundur2Areas')
mpc_reader_pf.load_mpc(domain=matpower.Domain.PF)
system_pf = mpc_reader_pf.system

# log results
logger = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
    logger.log_attribute(node.name()+'.S', 's', node)

# Parametrize and run simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.info)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(1)
sim_pf.set_final_time(0.1)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.add_logger(logger)
sim_pf.run()

In [ ]:
#system_pf

### Split topologie at node 8

In [ ]:
cosim_config = {
    "number_topologies": 2,
    "nodes": [["N1", "N2", "N5", "N6", "N7", "N8"], ["N3", "N4", "N11", "N10", "N9", "N8"]],
    "eq_component": ["VS", "CS"],
    "split_node": "N8"
}

#### Create topoplogies for cosimulation

In [ ]:
sim_name_fault = "Cosim_Kundur2Areas_Fault"        
dpsimpy.Logger.set_log_dir('logs/' + sim_name_fault)

# load dynamic topology
mpc_reader_dyn = matpower.Reader(mpc_file_path=local_file_static, mpc_name='Kundur2Areas',
                                 mpc_dyn_file_path=local_file_dynamic, mpc_dyn_name='Kundur2Areas_dyn')
mpc_reader_dyn.create_dpsim_objects(domain=matpower.Domain.EMT, frequency=60, 
                                 with_avr=False, with_tg=False, with_pss=False)

### Extend topology with 3ps fault
#sw = mpc_reader_dyn.add_3ph_faut("N7", switch_closed=1e-3, switch_open=1e18)

# create dpsim topology
mpc_reader_dyn.create_dpsim_topology()

#initialize node voltages using pf results
system_dyn = mpc_reader_dyn.system
system_dyn.init_with_powerflow(system_pf, dpsimpy.Domain.EMT)

#create topologies for cosimulation
[sys_topo_1, sys_topo_2] = mpc_reader_dyn.create_cosim_topologies(cosim_config)

#### Test fist sub topology

In [ ]:
sim_name_fault_top1 = "Cosim_Kundur2Areas_Fault_Sub1"        
dpsimpy.Logger.set_log_dir('logs/' + sim_name_fault_top1)

# log results
logger = dpsimpy.Logger(sim_name_fault_top1)
for node in sys_topo_1.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
#for gen_name in ["Gen_N1", "Gen_N2"]:
#    logger.log_attribute(gen_name+".Pe", 'Te', mpc_reader_dyn.dpsimpy_comp_dict[gen_name][0])

# Parametrize and run simulation
sim = dpsimpy.Simulation(sim_name_fault_top1, dpsimpy.LogLevel.info)
sim.set_system(sys_topo_1)
sim.set_time_step(1e-4)
sim.set_final_time(1)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.KLU)
sim.do_init_from_nodes_and_terminals(True)
sim.add_logger(logger)
sim.do_system_matrix_recomputation(True)
# add events
#sw_event_1 = dpsimpy.event.SwitchEvent(1, sw, True)
#sw_event_2 = dpsimpy.event.SwitchEvent(1.1, sw, False)
#sim.add_event(sw_event_1)
#sim.add_event(sw_event_2)
sim.run()

#### Plot Results fist sub topology

In [ ]:
dpsim_result_file = 'logs/' + sim_name_fault_top1 + '/' + sim_name_fault_top1 + '.csv'
ts_dpsim_emt_top1 = read_timeseries_csv(dpsim_result_file)

In [ ]:
#mpc_reader_dyn.system

In [ ]:
timestep_common = 1e-3
t_begin = 0.0
t_end = 1
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)
time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

#plot parameters
width = 12
height = 4

varname_dpsim = "N8.V"
dpsim_emt_values = ts_dpsim_emt_top1[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx]
    
plt.figure(figsize=(width, height))
#plt.subplot(1, 2, 1)
plt.plot(time, dpsim_emt_values, ':', label='EMT - DPsim')
plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.grid()
#plt.ylim([1.058, 1.062])
#plt.xlim([0.95, 1.2])
plt.xlabel("time (s)")
#plt.ylabel(varname_digSilent + " (pu)")

#### Test second sub topology

In [ ]:
sim_name_fault_top2 = "Cosim_Kundur2Areas_Fault_Sub2"        
dpsimpy.Logger.set_log_dir('logs/' + sim_name_fault_top2)

# log results
logger = dpsimpy.Logger(sim_name_fault_top2)
for node in sys_topo_2.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
#for gen_name in ["Gen_N1", "Gen_N2"]:
#    logger.log_attribute(gen_name+".Pe", 'Te', mpc_reader_dyn.dpsimpy_comp_dict[gen_name][0])

# Parametrize and run simulation
sim = dpsimpy.Simulation(sim_name_fault_top2, dpsimpy.LogLevel.info)
sim.set_system(sys_topo_2)
sim.set_time_step(1e-4)
sim.set_final_time(1)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.KLU)
sim.do_init_from_nodes_and_terminals(True)
sim.add_logger(logger)
sim.do_system_matrix_recomputation(True)
# add events
#sw_event_1 = dpsimpy.event.SwitchEvent(1, sw, True)
#sw_event_2 = dpsimpy.event.SwitchEvent(1.1, sw, False)
#sim.add_event(sw_event_1)
#sim.add_event(sw_event_2)
sim.run()

In [ ]:
dpsim_result_file = 'logs/' + sim_name_fault_top2 + '/' + sim_name_fault_top2 + '.csv'
ts_dpsim_emt_top2 = read_timeseries_csv(dpsim_result_file)

In [ ]:
timestep_common = 1e-3
t_begin = 0.0
t_end = 1
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)
time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

#plot parameters
width = 12
height = 4

varname_dpsim = "N8.V"
dpsim_emt_values = ts_dpsim_emt_top2[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx]
    
plt.figure(figsize=(width, height))
#plt.subplot(1, 2, 1)
plt.plot(time, dpsim_emt_values, ':', label='EMT - DPsim')
plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.grid()
#plt.ylim([1.058, 1.062])
#plt.xlim([0.95, 1.2])
plt.xlabel("time (s)")
#plt.ylabel(varname_digSilent + " (pu)")

### 3. Dynamic Simulation with fault at node 7 (Full topology)

#### Declare some functions

In [ ]:
def Kundur2Areas_Fault(domain="SP"):
    if domain=="SP":
        matpower_domain = matpower.Domain.SP
        dpsim_domain = dpsimpy.Domain.SP
    elif domain=="DP":
        matpower_domain = matpower.Domain.DP
        dpsim_domain = dpsimpy.Domain.DP
    else:
        domain="EMT"
        matpower_domain = matpower.Domain.EMT
        dpsim_domain = dpsimpy.Domain.EMT

    sim_name_fault = domain + "_Kundur2Areas_Fault"        
    dpsimpy.Logger.set_log_dir('logs/' + sim_name_fault)

    mpc_reader_dyn = matpower.Reader(mpc_file_path=local_file_static, mpc_name='Kundur2Areas',
                                     mpc_dyn_file_path=local_file_dynamic, mpc_dyn_name='Kundur2Areas_dyn')
    mpc_reader_dyn.create_dpsim_objects(domain=matpower_domain, frequency=60, 
                                     with_avr=False, with_tg=False, with_pss=False)

    ### Extend topology with 3ps fault
    sw = mpc_reader_dyn.add_3ph_faut("N7", switch_closed=1e-3, switch_open=1e18)
    
    # create dpsim topology
    mpc_reader_dyn.create_dpsim_topology()

    #initialize node voltages using pf results
    system_dyn = mpc_reader_dyn.system
    system_dyn.init_with_powerflow(system_pf, dpsim_domain)

    # log results
    logger = dpsimpy.Logger(sim_name_fault)
    for node in system_dyn.nodes:
        logger.log_attribute(node.name()+'.V', 'v', node)
    for gen_name in ["Gen_N1", "Gen_N2", "Gen_N3", "Gen_N3", "Gen_N4"]:
        logger.log_attribute(gen_name+".Pe", 'Te', mpc_reader_dyn.dpsimpy_comp_dict[gen_name][0])

    # Parametrize and run simulation
    sim = dpsimpy.Simulation(sim_name_fault, dpsimpy.LogLevel.info)
    sim.set_system(system_dyn)
    if domain=="SP":
        sim.set_time_step(1e-3)
    else:
        sim.set_time_step(1e-4)
    sim.set_final_time(10)
    sim.set_domain(dpsim_domain)
    sim.set_solver(dpsimpy.Solver.MNA)
    sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.KLU)
    sim.do_init_from_nodes_and_terminals(True)
    sim.add_logger(logger)
    sim.do_system_matrix_recomputation(True)

    # add events
    sw_event_1 = dpsimpy.event.SwitchEvent(1, sw, True)
    sw_event_2 = dpsimpy.event.SwitchEvent(1.1, sw, False)
    sim.add_event(sw_event_1)
    sim.add_event(sw_event_2)
    sim.run()
    
    return sim_name_fault

#### Dpsim Simulations

In [ ]:
sim_name_fault_dp = Kundur2Areas_Fault(domain="DP")

In [ ]:
sim_name_fault_emt = Kundur2Areas_Fault(domain="EMT")

### Read dynamic results

In [ ]:
from villas.dataprocessing.timeseries import TimeSeries as ts
import villas.dataprocessing.plottools as pt

dpsim_result_file = 'logs/' + sim_name_fault_dp + '/' + sim_name_fault_dp + '.csv'
ts_dpsim_dp = read_timeseries_csv(dpsim_result_file)

dpsim_result_file = 'logs/' + sim_name_fault_emt + '/' + sim_name_fault_emt + '.csv'
ts_dpsim_emt = read_timeseries_csv(dpsim_result_file)

### Load DigSilent Results

In [ ]:
if not os.path.exists('Kundur2Areas-data'):
    os.mkdir('Kundur2Areas-data')

url = 'https://github.com/martinmoraga/dpsim_data/raw/main/Kundur2Areas/PowerFactory/DigSilent_Kundur2Areas_Fault_N7_TimeStep_100uS_EMT.csv'
local_file = './Kundur2Areas-data/DigSilent_Kundur2Areas_Fault_N7_TimeStep_100uS_EMT.csv'
urllib.request.urlretrieve(url, local_file)
ts_digSilent = read_timeseries_dpsim(local_file)

### Define plot functions

In [ ]:
timestep_common = 1e-3
t_begin = 0.0
t_end = 10
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)
time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

#plot parameters
width = 12
height = 4

def plot_node_volt_abs(varname_dpsim, varname_digSilent, 
                       ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, nominal_voltage, timestep_common=1e-3, y_lim=False):
    
    #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    dpsim_dp_values_shifted = np.real(ts_dpsim_dp[varname_dpsim].interpolate(timestep_common).frequency_shift(60).values[begin_idx:end_idx] / nominal_voltage)
    dpsim_emt_values = ts_dpsim_emt[varname_dpsim+"_0"].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage * np.sqrt(3/2)
    digSilent_values = ts_digSilent[varname_digSilent].interpolate(timestep_common).values[begin_idx:end_idx]
    
    plt.figure(figsize=(width, height))
    plt.subplot(1, 2, 1)
    plt.plot(time, dpsim_dp_values_shifted, label='DP - DPsim')
    plt.plot(time, dpsim_emt_values, ':', label='EMT - DPsim')
    plt.plot(time, digSilent_values, '--', label='EMT - DigSilent')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.grid()
    #plt.ylim([1.058, 1.062])
    plt.xlim([0.95, 1.2])
    plt.xlabel("time (s)")
    plt.ylabel(varname_digSilent + " (pu)")
    if (y_lim):
        plt.set_ylim(y_lim)
    
    plt.subplot(1, 2, 2)
    plt.plot(time, dpsim_dp_values_shifted, label='DP - DPsim')
    plt.plot(time, dpsim_emt_values, ':', label='EMT - DPsim')
    plt.plot(time, digSilent_values, '--', label='EMT - DigSilent')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.grid()
    #plt.ylim([1.058, 1.062])
    plt.xlim([2.9, 3.1])
    plt.xlabel("time (s)")
    plt.ylabel(varname_digSilent + " (pu)")
    if (y_lim):
        plt.ylim(y_lim)
        
    plt.show()
    
    #calculate RMSE
    #rmse = np.sqrt(((dpsim_sp_values_abs_pu - psat_values) ** 2).mean())
    #print('RMSE {:s}  = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(varname_dpsim, rmse, rmse/1.0*100, 1.0))
    
def plot_elec_power(varname_dpsim, varname_digSilent, 
                    ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, timestep_common=1e-3, y_lim=False):
    
    #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    dpsim_dp_values = ts_dpsim_dp[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx] * 9
    dpsim_emt_values = ts_dpsim_emt[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx] * 9
    digSilent_values = ts_digSilent[varname_digSilent].interpolate(timestep_common).values[begin_idx:end_idx] / 100
    
    plt.figure(figsize=(width, height))
    plt.plot(time, dpsim_dp_values, label='DP - DPsim')
    plt.plot(time, dpsim_emt_values, '--', label='EMT - DPsim')
    plt.plot(time, digSilent_values, '--', label='EMT - DigSilent')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.grid()
    #plt.ylim([6.7, 7.1])
    plt.xlim([t_begin, t_end])
    plt.xlabel("time (s)")
    plt.ylabel(varname_dpsim + " (pu)")
    if (y_lim):
        plt.ylim(y_lim)
    plt.show()
    
    #calculate RMSE
    #rmse = np.linalg.norm(dpsim_sp_values - psat_values) / np.sqrt(len(psat_values))
    #print('RMSE {:s}  = {:.6f} (pu), which is {:.3f}% of the nominal value = ~{:.3f} (pu) '.format(varname_dpsim, rmse, rmse/7.0*100, 7.0))

    #return rmse

### Plot SG's electrical power

##### Gen1

In [ ]:
varname_dpsim = 'Gen_N1.Pe'
varname_psat = 'p_g3'
varname_digSilent = 'G1_p'
rmse_g1 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, timestep_common=1e-3, y_lim=[6.25,7.25])

##### Gen2

In [ ]:
varname_dpsim = 'Gen_N2.Pe'
varname_digSilent = 'G2_p'
rmse_g2 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, timestep_common=1e-3, y_lim=[6.4,7.2])

##### Gen3

In [ ]:
varname_dpsim = 'Gen_N3.Pe'
varname_digSilent = 'G4_p'
rmse_g3 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, timestep_common=1e-3, y_lim=[5,9])

##### Gen4

In [ ]:
varname_dpsim = 'Gen_N4.Pe'
varname_digSilent = 'G3_p'
rmse_g4 = plot_elec_power(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, timestep_common=1e-3, y_lim=[5.5,8.5])

### Assertion

In [ ]:
#assert(np.absolute(rmse_g1)<0.081)
#assert(np.absolute(rmse_g2)<0.094)
#assert(np.absolute(rmse_g3)<0.023)
#assert(np.absolute(rmse_g4)<0.024)

### Plot Voltages

#### LV side

In [ ]:
#"""
varname_dpsim = 'N1.V'
varname_digSilent = 'v1'
nominal_voltage = 20000
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, nominal_voltage, timestep_common=1e-3)
#"""

In [ ]:
#"""
varname_dpsim = 'N2.V'
varname_digSilent = 'v2'
nominal_voltage = 20000
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, nominal_voltage, timestep_common=1e-3)
#"""

In [ ]:
#"""
varname_dpsim = 'N3.V'
varname_digSilent = 'v3'
nominal_voltage = 20000
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, nominal_voltage, timestep_common=1e-3)
#"""

In [ ]:
#"""
varname_dpsim = 'N4.V'
varname_digSilent = 'v4'
nominal_voltage = 20000
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, nominal_voltage, timestep_common=1e-3)
#"""

#### Low voltage side

In [ ]:
#"""
varname_dpsim = 'N5.V'
varname_digSilent = 'v5'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, nominal_voltage, timestep_common=1e-3)
#"""


In [ ]:
#"""
varname_dpsim = 'N6.V'
varname_digSilent = 'v6'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, nominal_voltage, timestep_common=1e-3)
#"""

In [ ]:
#"""
varname_dpsim = 'N7.V'
varname_digSilent = 'v7'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, nominal_voltage, timestep_common=1e-3)
#"""

In [ ]:
#"""
varname_dpsim = 'N8.V'
varname_digSilent = 'v8'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, nominal_voltage, timestep_common=1e-3)
#"""

In [ ]:
#"""
varname_dpsim = 'N9.V'
varname_digSilent = 'v9'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, nominal_voltage, timestep_common=1e-3)
#"""

In [ ]:
#"""
varname_dpsim = 'N10.V'
varname_digSilent = 'v10'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, nominal_voltage, timestep_common=1e-3)
#"""

In [ ]:
#"""
varname_dpsim = 'N11.V'
varname_digSilent = 'v11'
nominal_voltage = 230000
plot_node_volt_abs(varname_dpsim, varname_digSilent, ts_dpsim_emt, ts_dpsim_dp, ts_digSilent, nominal_voltage, timestep_common=1e-3)
#"""